In [91]:
# Import packages
import requests
import pandas as pd
import glob # File path package 

In [120]:
# Call Etherscan API 
contract_add = '0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb' # VeeFriends Contract Address 
address = "0x6abd9b8ac8797c4227868cef0b1e15d1a1aff48e" # Example

#url = "http://api.etherscan.io/api?module=account&action=tokennfttx&contractaddress=" + contract_add + "&page=1&offset=100&sort=asc&apikey=YourApiKeyToken"
url = "http://api.etherscan.io/api?module=account&action=tokennfttx&contractaddress=" + contract_add + "&address=" + address + "&page=1&offset=100&sort=asc&apikey=YourApiKeyToken"


In [121]:
response = requests.get(url)
address_content = response.json()
address_content

{'status': '1',
 'message': 'OK-Missing/Invalid API Key, rate limit of 1/5sec applied',
 'result': [{'blockNumber': '12421047',
   'timeStamp': '1620842799',
   'hash': '0xca047b9b577c805a0bc919f9d067b9b3665d1ee846c05911a5c9aec50c081f17',
   'nonce': '2',
   'blockHash': '0x21176d166141ce7e379b88c2348064b8907c3d938a6799a09f1c0531ac63afb2',
   'from': '0x0000000000000000000000000000000000000000',
   'contractAddress': '0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb',
   'to': '0x6abd9b8ac8797c4227868cef0b1e15d1a1aff48e',
   'tokenID': '8593',
   'tokenName': 'VeeFriends',
   'tokenSymbol': 'VFT',
   'tokenDecimal': '0',
   'transactionIndex': '309',
   'gas': '280000',
   'gasPrice': '497000000000',
   'gasUsed': '264532',
   'cumulativeGasUsed': '13693819',
   'input': 'deprecated',
   'confirmations': '1260169'},
  {'blockNumber': '12421052',
   'timeStamp': '1620842846',
   'hash': '0xe504f9a19ecd54bdf07d281436b3eee6b59713696534d2b727249b964cd41151',
   'nonce': '3',
   'blockHash': '0x9

<h3><center> Obtain Addresses

In [94]:
path = r'/Users/loomanke/Documents/Github/Personal/NFT_Project/DATA' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

# Read in and concat token address csv files 
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [95]:
# Limit data to token address holders 
token_owners = frame["From"].unique()

In [66]:
def divide_chunks(l, n):
    '''
    Input: list of token addresses 
    Output: nested list with 100 addresses in each list
    '''
      
    # looping till length l
    for i in range(0, len(l), n): 
        yield l[i:i + n]

n = 100
divided_token_owners = list(divide_chunks(token_owners, n))
#print(divided_token_owners)

<h3><center> Testing

Goal: Reading in the full list of addresses exceeds the limit of my API call. I subset the data into groups of 100 (nested list). I would like to read in each list object separately so as to not exceed my call limit. 

In [126]:
import random
import time
sleep = 30

apiToken = '6GC1UPX46NE1QPVI9YKI9IPJMFVF6MHKM2' # API token 
eventTransfer = [] # Storing transacation data 
contract_add = '0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb' # VeeFriends Contract Address 

for token_owner in divided_token_owners: 
    for owner in token_owner:
        try: 
            address = str(owner)
            url = "http://api.etherscan.io/api?module=account&action=tokennfttx&contractaddress=" + contract_add + "&address=" + address + "&page=1&offset=100&sort=asc&apikey=" + apiToken
            response = requests.get(url)
            address_content = response.json()
            eventTransfer.append(address_content)
        except requests.exceptions.ConnectionError:
            r.status_code = "Connection refused"
    time.sleep(random.uniform(0, sleep))

In [128]:
eventTransfer = pd.DataFrame(eventTransfer)
eventTransfer

status message  \
0     1      OK       
1     1      OK       
2     1      OK       
3     1      OK       
4     1      OK       
...  ..      ..       
2867  0      NOTOK    
2868  0      NOTOK    
2869  1      OK       
2870  1      OK       
2871  1      OK       

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [129]:
pd.set_option('display.max_colwidth', -1) # or 199.
eventTransfer['message']. value_counts()

0       OK   
1       OK   
2       OK   
3       OK   
4       OK   
        ..   
2867    NOTOK
2868    NOTOK
2869    OK   
2870    OK   
2871    OK   
Name: message, Length: 2872, dtype: object

In [148]:
# Filter only OK messages for now 
eventTransfer_OK = eventTransfer[eventTransfer['message'] == 'OK']
eventTransfer_OK['message'].value_counts()

OK    1570
Name: message, dtype: int64

In [311]:
# Loop through data to transform it to a pd df 
df = pd.DataFrame()
for x in range(len(eventTransfer_OK)): 
    trial = eventTransfer_OK['result'][x]
    df_1 = pd.DataFrame(trial) 
    df = df.append(df_1)

In [312]:
# Reset index 
df = df.reset_index()
# Drop unecessary columns 
df = df.drop('index', 1)
df = df.drop('level_0', 1)